In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\naura\\Desktop\\end-to-end-ML\\computer-vision\\Bone-fracture-xray-classification'

In [2]:
import mlflow
from dotenv import dotenv_values

In [3]:
dagshub = dotenv_values(".env")
MLFLOW_TRACKING_URI = dagshub.get("MLFLOW_TRACKING_URI")
MLFLOW_TRACKING_USERNAME = dagshub.get("MLFLOW_TRACKING_USERNAME")
MLFLOW_TRACKING_PASSWORD = dagshub.get("MLFLOW_TRACKING_PASSWORD")

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/custom_train_model.h5")

In [6]:
from pydantic import BaseModel
from pathlib import Path

class InferenceConfig(BaseModel):
    path_to_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from boneFractureClassification.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from boneFractureClassification.utils.common import read_yaml, create_directories, save_json

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_inference_config(self) -> InferenceConfig:
        inference_config = InferenceConfig(
            path_to_model="artifacts/training/custom_train_model.h5",
            training_data="artifacts/data_ingestion/data.zip/Bone_Fracture_Binary_Classification/test",
            mlflow_uri=MLFLOW_TRACKING_URI,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return inference_config

In [9]:
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Inference:
    def __init__(self, config: InferenceConfig):
        self.config = config
        
    def _train_val_data_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod    
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def inference(self):
        self.model = self.load_model(self.config.path_to_model)
        self._train_val_data_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri=MLFLOW_TRACKING_URI
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16_finetuned")
            else:
                mlflow.keras.log_model(self.model, "model")
        
        
        

In [19]:
try:
    config = ConfigurationManager()
    inference_config = config.get_inference_config()
    inference = Inference(inference_config)
    inference.inference()
    inference.log_into_mlflow()
except Exception as e:
    raise e
    

[2024-06-06 17:09:25,079: INFO: common: yaml file: config\config.yaml loaded successfully!]
[2024-06-06 17:09:25,083: INFO: common: yaml file: params.yaml loaded successfully!]
[2024-06-06 17:09:25,087: INFO: common: created directory at: artifacts]
Found 151 images belonging to 2 classes.
10/10 [==============================] - 139s 7s/step - loss: 34.5226 - accuracy: 0.5629
[2024-06-06 17:11:46,187: INFO: common: json file saved at: scores.json]


2024/06/06 17:11:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-06-06 17:11:50,384: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\naura\AppData\Local\Temp\tmpgc0_abow\model\data\model\assets
[2024-06-06 17:11:51,532: INFO: builder_impl: Assets written to: C:\Users\naura\AppData\Local\Temp\tmpgc0_abow\model\data\model\assets]


c:\Users\naura\anaconda3\envs\bone-fracture\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2024-06-06 17:12:26,928: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /nilesh.auradkar.0599/Bone-fracture-xray-classification.mlflow/api/2.0/mlflow/runs/get?run_uuid=25f72904552f4c3bad76fc6fde28f731&run_id=25f72904552f4c3bad76fc6fde28f731]
[2024-06-06 17:12:34,141: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)': /nilesh.auradkar.0599/Bone-fracture-xray-classification.mlflow/api/2.0/mlflow-artifacts/artifacts/1a698a69f1d34d34b4893427b01ebd41/25f72904552f4c3bad76fc6fde28f731/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2024-06-06 17:13:07,117: WARNING: connectionpool: Retrying (Re

Successfully registered model 'VGG16_finetuned'.
2024/06/06 17:14:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16_finetuned, version 1
Created version '1' of model 'VGG16_finetuned'.


In [1]:
from datetime import datetime

In [5]:
current_datetime = datetime.now().strftime("%Y-%m-%d%H:%M:%S")
current_datetime = current_datetime.replace()

'2024-06-0621:16:33'

In [11]:
import os
filename = f"model_{str(current_datetime)}.h5"

In [13]:
os.makedirs(filename)

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'model_2024-06-0621:16:33.h5'

In [17]:
def select_recent_model():
    search_dir_path = "./artifacts/training/"
    recent_model = None
    most_recent_time = 0
    for file in os.scandir(search_dir_path):
        modify_time = file.stat().st_mtime_ns
        if modify_time > most_recent_time:
            recent_model = file.name
            most_recent_time = modify_time

    return recent_model

In [20]:
import os
os.chdir("../")
%pwd

'c:\\Users\\naura\\Desktop\\end-to-end-ML\\computer-vision\\Bone-fracture-xray-classification'

In [21]:
recent_model = select_recent_model()
recent_model

'custom_model_train_2024-06-06_22-14-48.h5'